In [1]:
import pandas as pd

In [2]:
file_path ='itmo_sber.csv'
data = pd.read_csv('itmo_sber.csv')

In [3]:
print(data.isnull().sum())

Unnamed: 0                    0
Задача                        0
Обстановка                    0
Оптимальный план              0
Задача en                     0
Обстановка en                 0
Оптимальный план en           0
Предсказанный план            0
Успех предсказанного плана    0
dtype: int64


In [4]:
print(data.head())

   Unnamed: 0                          Задача  \
0           0   Приготовить горячий шоколад.    
1           1     Заказать доставку еды домой   
2           2  Реставрация старого комода\r\n   
3           3            Приготовить сырники.   
4           4               Нарезать колбасу.   

                                          Обстановка  \
0  кухня, молоко, какао, сахар, кастрюля, ложка, ...   
1  мобильный телефон, ресторанное меню, интернет,...   
2  Старый комод, повреждения, шпатель, шкурка, ма...   
3  Кухня, столешница, творог, мука, яйцо, сахар, ...   
4  Кухня, колбаса, нож, доска для резки, холодиль...   

                                    Оптимальный план  \
0  Откройте дверь. Зайдите на кухню. Закрйоте две...   
1  Найдите стол. Подойдите к столу. Найдите мобил...   
2  Подойдите к комоду.Найдите шкурку.Возьмите шку...   
3  Подойти к столешнице. Найти творог. Взять твор...   
4  Подойти к холодильнику. Открыть холодильник. Н...   

                               

In [5]:
print(data['Успех предсказанного плана'].value_counts())

Успех предсказанного плана
1    3234
0    1600
Name: count, dtype: int64


In [6]:
print(data.describe())

        Unnamed: 0  Успех предсказанного плана
count  4834.000000                 4834.000000
mean   2416.500000                    0.669011
std    1395.599931                    0.470618
min       0.000000                    0.000000
25%    1208.250000                    0.000000
50%    2416.500000                    1.000000
75%    3624.750000                    1.000000
max    4833.000000                    1.000000


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Задача'])

In [9]:
print(X.shape)

(4834, 5402)


In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words and word.isalpha()]
    return ' '.join(words)

data['Задача'] = data['Задача'].apply(preprocess_text)
data['Обстановка'] = data['Обстановка'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
# # Инициализация векторизатора
# tfidf_vectorizer = TfidfVectorizer(max_features=5000)
#
# # Объединяем текстовые данные для векторизации
# combined_text = data['Задача'] + " " + data['Обстановка']
# X = tfidf_vectorizer.fit_transform(combined_text)
#
# # Целевая переменная
# y = data['Успех предсказанного плана']

In [14]:
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# import re
#
# # Объединение текстовых полей
# data['combined_text'] = data['Задача en'] + ' ' + data['Обстановка en'] + ' ' + data['Предсказанный план']
#
# # Функция для очистки и нормализации текста
# def clean_text(text):
#     text = text.lower()  # Приведение к нижнему регистру
#     text = re.sub(r'\s+', ' ', text)  # Удаление лишних пробелов
#     text = re.sub(r'[^\w\s]', '', text)  # Удаление специальных символов
#     return text
#
# # Применение функции очистки к каждой строке объединенного текстового поля
# data['cleaned_text'] = data['combined_text'].apply(clean_text)
#
# # Подготовка данных и целевой переменной
# X = data['cleaned_text']
# y = data['Успех предсказанного плана']
#
# # Разделение данных на обучающую и тестовую выборки
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#
# # Векторизация текста с использованием TF-IDF
# tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # Ограничение на количество признаков для упрощения модели
# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# X_test_tfidf = tfidf_vectorizer.transform(X_test)
#
# X_train_tfidf.shape, X_test_tfidf.shape


((3867, 1000), (967, 1000))

In [15]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Загрузка предобученной модели BERT и токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

print('AAA')
# Предварительная обработка текста
def preprocess_text(text):
    text = text.lower()
    text = text.strip()
    return text

data['Предсказанный план'] = data['Предсказанный план'].apply(preprocess_text)
data['Оптимальный план'] = data['Оптимальный план'].apply(preprocess_text)

n=0
# Функция для получения эмбеддингов текста с помощью BERT
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    global n
    n=n+1
    print(n)
    return embeddings

# Получение эмбеддингов для "Предсказанного плана" и "Оптимального плана"
predicted_plan_embeddings = np.array([get_bert_embeddings(text) for text in data['Предсказанный план']])
optimal_plan_embeddings = np.array([get_bert_embeddings(text) for text in data['Оптимальный план']])

# Вычисление сходства текстов с помощью косинусного сходства
cosine_similarities = [np.dot(pred, opt.T) / (np.linalg.norm(pred) * np.linalg.norm(opt)) for pred, opt in zip(predicted_plan_embeddings, optimal_plan_embeddings)]

# Подготовка данных и целевой переменной для классификации
X = np.array(cosine_similarities).reshape(-1, 1)
y = data['Успех предсказанного плана']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


AAA
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


In [16]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# Обучение классификатора
model = LogisticRegression()
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the model: {accuracy}')

# Вычисление F1-меры
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

Accuracy of the model: 0.6608066184074457
F1 Score: 0.80


# OPTUNA log reg

In [25]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

def objective(trial):
    # Определите пространство поиска гиперпараметров
    C = trial.suggest_float("C", 0.0001, 10000)
    penalty = trial.suggest_categorical("penalty", ["l2", "none"])
    max_iter = trial.suggest_int("max_iter", 100, 1000)

    # Модель логистической регрессии
    model = LogisticRegression(
        C=C,
        penalty=penalty,
        max_iter=max_iter
    )

    # Обучение
    model.fit(X_train, y_train)

    # Предсказание
    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred)

    return f1

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=2000)

#лучшие гиперпараметры
best_params = study.best_params
best_C = best_params["C"]
best_penalty = best_params["penalty"]
best_max_iter = best_params["max_iter"]

#модель с лучшими гиперпараметрами
best_model = LogisticRegression(
    C=best_C,
    penalty=best_penalty,
    max_iter=best_max_iter
)


best_model.fit(X, y)

y_pred=best_model.predict(X_test)

[I 2024-04-04 15:21:00,139] A new study created in memory with name: no-name-34f8161c-aa96-4066-bbab-21fa4f442127
C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
[I 2024-04-04 15:21:00,156] Trial 0 finished with value: 0.7957658779576587 and parameters: {'C': 8026.730016517449, 'penalty': 'none', 'max_iter': 160}. Best is trial 0 with value: 0.7957658779576587.
C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the pas

In [26]:
#Лучшие гиперпараметры для логистической регрессии
print(best_params)

# Оценим качество
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc:.2f}")

{'C': 8026.730016517449, 'penalty': 'none', 'max_iter': 160}
Accuracy: 0.66
F1 Score: 0.80
ROC-AUC Score: 0.50
